In [ ]:
!pip install tensorflow opencv-python matplotlib

In [ ]:
!pip install tf-keras

In [ ]:
!pip list

In [ ]:
import tensorflow as tf
import os

In [ ]:
# Avoid OOM errors by setting GPU Memory Consumption Growth
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus: 
    tf.config.experimental.set_memory_growth(gpu, True)

In [ ]:
tf.config.list_physical_devices('GPU')

In [ ]:
import cv2
import imghdr

In [ ]:
data_dir = 'data' 

In [ ]:
image_exts = ['jpeg','jpg', 'bmp', 'png']

In [ ]:
for image_class in os.listdir(data_dir): 
    for image in os.listdir(os.path.join(data_dir, image_class)):
        image_path = os.path.join(data_dir, image_class, image)
        try: 
            img = cv2.imread(image_path)
            tip = imghdr.what(image_path)
            if tip not in image_exts: 
                print('Image not in ext list {}'.format(image_path))
                os.remove(image_path)
        except Exception as e: 
            print('Issue with image {}'.format(image_path))
            # os.remove(image_path)

In [ ]:
import numpy as np
from matplotlib import pyplot as plt

In [ ]:
data = tf.keras.utils.image_dataset_from_directory('data')

In [ ]:
data_iterator = data.as_numpy_iterator()

In [ ]:
batch = data_iterator.next()

In [ ]:
fig, ax = plt.subplots(ncols=4, figsize=(20,20))
for idx, img in enumerate(batch[0][:4]):
    ax[idx].imshow(img.astype(int))
    ax[idx].title.set_text(batch[1][idx])

In [ ]:
batch[0].shape

In [ ]:
#Data Augmentation
data_augmentation = tf.keras.Sequential([
    tf.keras.layers.RandomFlip('horizontal_and_vertical'),
    tf.keras.layers.RandomRotation(0.2),
    tf.keras.layers.RandomZoom(0.2, 0.2)
])

data = data.map(lambda x, y: (data_augmentation(x), y))

In [ ]:
data = data.map(lambda x,y: (x/255, y))

In [ ]:
data.as_numpy_iterator().next()

In [ ]:
batch[0].max()

In [ ]:
train_size = int(len(data)*.7)
val_size = int(len(data)*.2)
test_size = int(len(data)*.1)

In [ ]:
train_size

In [ ]:
train = data.take(train_size)
val = data.skip(train_size).take(val_size)
test = data.skip(train_size+val_size).take(test_size)

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Flatten, Dropout

In [ ]:
model = Sequential()

In [ ]:
model.add(Conv2D(16, (3,3), 1, activation='relu', input_shape=(256,256,3)))
model.add(MaxPooling2D())
model.add(Conv2D(32, (3,3), 1, activation='relu'))
model.add(MaxPooling2D())
model.add(Conv2D(16, (3,3), 1, activation='relu'))
model.add(MaxPooling2D())
model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

In [ ]:
model.compile('adam', loss=tf.losses.BinaryCrossentropy(), metrics=['accuracy'])

In [ ]:
model.summary()

In [ ]:
#Regularization
#model.add(tf.keras.layers.Dense(256, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.01)))
#model.add(tf.keras.layers.Dropout(0.2))

In [ ]:
logdir='logs'

In [ ]:
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=logdir)

In [ ]:
hist = model.fit(train, epochs=20, validation_data=val, callbacks=[tensorboard_callback])

In [ ]:
#Hyperparameter Tuning
from tf.keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import GridSearchCV

param_grid = {'batch_size': [32, 64, 128], 'epochs': [10, 20, 30]}
grid_search = GridSearchCV(KerasClassifier(build_fn=model), param_grid, cv=5)
grid_search.fit(train)
print(grid_search.best_params_)
print(grid_search.best_score_)

In [ ]:
#Ensemble Methods
from tf.keras.ensemble import VotingClassifier

model1 = tf.keras.Model(inputs=base_model.input, outputs=x)
model2 = tf.keras.Model(inputs=base_model.input, outputs=x)
model3 = tf.keras.Model(inputs=base_model.input, outputs=x)

ensemble_model = VotingClassifier(estimators=[model1, model2, model3], voting='soft')

In [ ]:
fig = plt.figure()
plt.plot(hist.history['loss'], color='teal', label='loss')
plt.plot(hist.history['val_loss'], color='orange', label='val_loss')
fig.suptitle('Loss', fontsize=20)
plt.legend(loc="upper left")
plt.show()

In [ ]:
fig = plt.figure()
plt.plot(hist.history['accuracy'], color='teal', label='accuracy')
plt.plot(hist.history['val_accuracy'], color='orange', label='val_accuracy')
fig.suptitle('Accuracy', fontsize=20)
plt.legend(loc="upper left")
plt.show()

In [ ]:
from tensorflow.keras.metrics import Precision, Recall, BinaryAccuracy

In [ ]:
pre = Precision()
re = Recall()
acc = BinaryAccuracy()

In [ ]:
for batch in test.as_numpy_iterator(): 
    X, y = batch
    yhat = model.predict(X)
    pre.update_state(y, yhat)
    re.update_state(y, yhat)
    acc.update_state(y, yhat)

In [ ]:
print(f'Precision:{pre.result().numpy()}, Recall:{re.result().numpy()}, Accuracy:{acc.result().numpy()}')

In [ ]:
import cv2

In [ ]:
img = cv2.imread('Normal test 1 (10).png')
plt.imshow(img)
plt.show()

In [ ]:
resize = tf.image.resize(img, (256,256))
plt.imshow(resize.numpy().astype(int))
plt.show()

In [ ]:
yhat = model.predict(np.expand_dims(resize/255, 0))

In [ ]:
yhat

In [ ]:
if yhat > 0.5: 
    print(f'Predicted class is Normal')
else:
    print(f'Predicted class is Malignant')

In [ ]:
from sklearn.metrics import precision_score, recall_score, f1_score

pre = Precision()
re = Recall()
acc = BinaryAccuracy()

for batch in test.as_numpy_iterator():
    X, y = batch
    yhat = model.predict(X)
    pre.update_state(y, yhat)
    re.update_state(y, yhat)
    acc.update_state(y, yhat)

print(f'Precision: {pre.result().numpy()}')
print(f'Recall: {re.result().numpy()}')
print(f'Accuracy: {acc.result().numpy()}')

# Additional Evaluation using sklearn
# Flatten the arrays to compare each pixel/element
y_pred = np.concatenate([model.predict(batch[0]) for batch in test], axis=0)
y_true = np.concatenate([batch[1] for batch in test], axis=0)

# Binarize predictions
y_pred_binary = (y_pred > 0.5).astype(int)

# Calculate metrics
precision = precision_score(y_true, y_pred_binary)
recall = recall_score(y_true, y_pred_binary)
f1 = f1_score(y_true, y_pred_binary)

print(f'Sklearn Precision: {precision}')
print(f'Sklearn Recall: {recall}')
print(f'Sklearn F1 Score: {f1}')

metrics_history = {'precision': [], 'recall': [], 'f1': []}

for epoch in range(10):  # or however many iterations you want
    #... (rest of the code remains the same)

    # Calculate metrics
    precision = precision_score(y_true, y_pred_binary)
    recall = recall_score(y_true, y_pred_binary)
    f1 = f1_score(y_true, y_pred_binary)

    metrics_history['precision'].append(precision)
    metrics_history['recall'].append(recall)
    metrics_history['f1'].append(f1)

    print(f'Epoch {epoch+1}, Precision: {precision:.4f}, Recall: {recall:.4f}, F1 Score: {f1:.4f}')

# Visualize metrics
fig, ax = plt.subplots(1, 3, figsize=(15, 5))

ax[0].plot(metrics_history['precision'])
ax[0].set_title('Precision Over Time')
ax[0].set_xlabel('Epoch')
ax[0].set_ylabel('Value')
ax[0].set_ylim([0, 1])

ax[1].plot(metrics_history['recall'])
ax[1].set_title('Recall Over Time')
ax[1].set_xlabel('Epoch')
ax[1].set_ylabel('Value')
ax[1].set_ylim([0, 1])

ax[2].plot(metrics_history['f1'])
ax[2].set_title('F1 Score Over Time')
ax[2].set_xlabel('Epoch')
ax[2].set_ylabel('Value')
ax[2].set_ylim([0, 1])

plt.show()

In [ ]:
import matplotlib.pyplot as plt

#... (rest of the code remains the same)

# Calculate metrics
precision = precision_score(y_true, y_pred_binary)
recall = recall_score(y_true, y_pred_binary)
f1 = f1_score(y_true, y_pred_binary)

print(f'Sklearn Precision: {precision}')
print(f'Sklearn Recall: {recall}')
print(f'Sklearn F1 Score: {f1}')

# Visualize metrics
fig, ax = plt.subplots(1, 3, figsize=(15, 5))

ax[0].bar(['Precision'], [precision])
ax[0].set_title('Precision')
ax[0].set_ylabel('Value')
ax[0].set_ylim([0, 1])

ax[1].bar(['Recall'], [recall])
ax[1].set_title('Recall')
ax[1].set_ylabel('Value')
ax[1].set_ylim([0, 1])

ax[2].bar(['F1 Score'], [f1])
ax[2].set_title('F1 Score')
ax[2].set_ylabel('Value')
ax[2].set_ylim([0, 1])

plt.show()

In [ ]:
from tensorflow.keras.models import load_model

In [ ]:
model.save(os.path.join('models','imageclassifier.h5'))

In [ ]:
new_model = load_model('imageclassifier.h5')

In [ ]:
new_model.predict(np.expand_dims(resize/255, 0))